In [ ]:
!pip install transformers

In [ ]:
!pip install pytorch-lightning

In [ ]:
import numpy as np
import pandas as pd

import torch
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.core.lightning import LightningModule
from torch.utils.data import DataLoader, Dataset

from transformers.optimization import AdamW, get_cosine_schedule_with_warmup
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel

import re
import math
import random
from tqdm import tqdm

## (1) 데이터 전처리
문장부호 유지

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/데이터/인물 챗봇/동룡_최종.csv")

In [ ]:
df.shape

(17434, 4)

In [ ]:
df['qa'] = df['Q'] + df['A']

In [ ]:
df.drop_duplicates(subset='qa', inplace = True, ignore_index=True )

In [ ]:
df.head(5)

,Unnamed: 0,Q,A,role,qa
0,0,ㅁ..미안,ㅁ..미안,동룡,ㅁ..미안ㅁ..미안
1,2,미안하다..,ㅁ..미안,동룡,미안하다..ㅁ..미안
2,4,죄송함다…,ㅁ..미안,동룡,죄송함다…ㅁ..미안
3,10,나가서 걸레 좀 갖고와.,"좀 부엌에 술빵 남은 거 있드라, 그것도",동룡,"나가서 걸레 좀 갖고와.좀 부엌에 술빵 남은 거 있드라, 그것도"
4,11,나가서 걸레 좀 갖고와.,"부엌에 남은 술빵 거 있드라, 그것도 좀",동룡,"나가서 걸레 좀 갖고와.부엌에 남은 술빵 거 있드라, 그것도 좀"


In [ ]:
df.shape

(14245, 5)

In [ ]:
df.drop(['Unnamed: 0', 'role', 'qa'], inplace = True, axis = 1)

## (2) 모델링
GPU

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
                                                    bos_token='</s>', eos_token='</s>', unk_token='<unk>',pad_token='<pad>', mask_token='<mask>')
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

Downloading:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


Downloading:   0%|          | 0.00/513M [00:00<?, ?B/s]

In [ ]:
# 챗봇 데이터를 처리하는 클래스를 만든다.
class ChatbotDataset(Dataset):
    def __init__(self, chats, max_len=40):  # 데이터셋의 전처리를 해주는 부분
        self._data = chats
        self.max_len = max_len
        self.q_token = "<usr>"
        self.a_token = "<sys>"
        self.bos = '</s>'
        self.eos = '</s>'
        self.pad = '<pad>'
        self.mask = '<mask>'
        self.tokenizer = tokenizer

    def __len__(self):  # chatbotdata 의 길이를 리턴한다.
        return len(self._data)

    def __getitem__(self, idx):  # 로드한 챗봇 데이터를 차례차례 DataLoader로 넘겨주는 메서드
        turn = self._data.iloc[idx]
        q = turn["Q"]  # 질문 열을 가져온다.
        a = turn["A"]  # 답변 열을 가져온다.

        q_toked = self.tokenizer.tokenize(self.q_token + q  + self.bos)
        q_len = len(q_toked)

        a_toked = self.tokenizer.tokenize(self.a_token + a + self.eos)
        a_len = len(a_toked)

        #질문의 길이가 최대길이보다 크면
        if q_len > self.max_len:
            a_len = self.max_len - q_len        # 최대길이 - 질문길이
            if a_len <= 0:       # 질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   # 질문길이를 최대길이의 반으로 
                q_len = len(q_toked)
                a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        #질문의 길이 + 답변의 길이가 최대길이보다 크면
        if q_len + a_len > self.max_len:
            a_len = self.max_len - q_len        #답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로 
                q_len = len(q_toked)
                a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        # 답변 labels = [mask, mask, ...., mask, ..., <bos>,..답변.. <eos>, <pad>....]
        labels = [self.mask,] * q_len + a_toked[1:]

        # mask = 질문길이 0 + 답변길이 1 + 나머지 0
        mask = [0] * q_len + [1] * a_len + [0] * (self.max_len - q_len - a_len)
        # 답변 labels을 index 로 만든다.
        labels_ids = self.tokenizer.convert_tokens_to_ids(labels)
        # 최대길이만큼 PADDING
        while len(labels_ids) < self.max_len:
            labels_ids += [self.tokenizer.pad_token_id]

        # 질문 + 답변을 index 로 만든다.    
        token_ids = self.tokenizer.convert_tokens_to_ids(q_toked + a_toked)
        # 최대길이만큼 PADDING
        while len(token_ids) < self.max_len:
            token_ids += [self.tokenizer.pad_token_id]

        # 질문+답변, 마스크, 답변
        return (token_ids, np.array(mask), labels_ids)

In [ ]:
def collate_batch(batch):
    data = [item[0] for item in batch]
    mask = [item[1] for item in batch]
    label = [item[2] for item in batch]
    return torch.LongTensor(data).to(device), torch.LongTensor(mask).to(device), torch.LongTensor(label).to(device)

In [ ]:
train_set = ChatbotDataset(df, max_len=40) 
train_dataloader = DataLoader(train_set, batch_size=32, num_workers=0, shuffle=True, collate_fn=collate_batch,)

In [ ]:
model  = model.to(device)
model.train()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [ ]:
learning_rate = 3e-5
criterion = torch.nn.CrossEntropyLoss(reduction="none")
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

epoch = 10
Sneg = -1e18

In [ ]:
for epoch in tqdm(range(epoch)):
    for batch_idx, samples in enumerate(train_dataloader):
        optimizer.zero_grad()
        token_ids, mask, label = samples
        out = model(token_ids)
        out = out.logits      #Returns a new tensor with the logit of the elements of input
        mask_3d = mask.unsqueeze(dim=2).repeat_interleave(repeats=out.shape[2], dim=2)
        mask_out = torch.where(mask_3d == 1, out, Sneg * torch.ones_like(out))
        loss = criterion(mask_out.transpose(2, 1), label)
        # 평균 loss 만들기 avg_loss[0] / avg_loss[1] <- loss 정규화
        avg_loss = loss.sum() / mask.sum()
        avg_loss.backward()
        # 학습 끝
        optimizer.step()
print ("end")

  0%|          | 0/10 [00:00<?, ?it/s]<ipython-input-14-d9c5350f11d2>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  return torch.LongTensor(data).to(device), torch.LongTensor(mask).to(device), torch.LongTensor(label).to(device)
100%|██████████| 10/10 [28:46<00:00, 172.69s/it]

end


30분

In [ ]:
torch.save(model, '/content/drive/MyDrive/데이터/인물 챗봇/동룡kogpt2.pt')

##(3) 챗봇 결과
NONE

In [ ]:
!pip install transformers

In [ ]:
import torch

from transformers.optimization import AdamW, get_cosine_schedule_with_warmup
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [ ]:
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
                                                    bos_token='</s>', eos_token='</s>', unk_token='<unk>',pad_token='<pad>', mask_token='<mask>')
model = torch.load('/content/drive/MyDrive/데이터/인물 챗봇/동룡kogpt2.pt', map_location=device)

Downloading:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [ ]:
with torch.no_grad():
    while 1:
        q = input("user > ").strip()
        if q == "quit":
            break
        a = ""
        while 1:
            input_ids = torch.LongTensor(tokenizer.encode("<usr>" + q +"<sys>" + a)).unsqueeze(dim=0)
            input_ids.to(device)
            pred = model(input_ids)
            pred = pred.logits
            gen = tokenizer.convert_ids_to_tokens(torch.argmax(pred, dim=-1).squeeze().numpy().tolist())[-1]
            if gen == '</s>':
                break
            a += gen.replace("▁", " ")
        print("Chatbot > {}".format(a.strip()))

user > 안녕
Chatbot > 굿바이 잘가~ 굿바이 마이 프렌드~
user > 너 뭐해
Chatbot > 뭐 하는거야 밤에, 하유 진짜...
user > 공부해야지
Chatbot > 학주..아니아니 니네 아부지 화학<unk>이게 전공이 뭐지
user > 나도 몰라
Chatbot > 있어서. 식구들이 너무 잘
user > 학교가자
Chatbot > 이게 같은 동네에 한 20년이나 살다보면, 이게 텔레파시가 우하하하
user > 텔레파시가 통했다
Chatbot > 야. 텔레파시가 우하하
user > 웃지마
Chatbot > 어머님!! 웃하십니까,
user > 나 진지해
Chatbot > 진지해. 알아, 나도
user > 좋아해
Chatbot > 너한테 반한 거 같애
user > 나 좋아해?
Chatbot > 나 너한테 반한 거 같애
user > 두근두근
Chatbot > 나두 근데  뭐가 거야 반갑다는 그럴리가 없잖아
user > 반가워
Chatbot > <unk>~
user > 배고파
Chatbot > 1,2번 보고 나가서 라면 하나 때리러 가자. 그럼 간다 하!
user > 라면 끓이자
Chatbot > 라면 이모, 3개요.
user > 정환이는 어디있어
Chatbot > 비디오를 뭔  보겠다고..
user > 춥다
Chatbot > 너 눈이 너무 거 높은 아니냐?
user > 추워
Chatbot > 너 눈이 너무 거 높은 아니냐?
user > 지루해
Chatbot > 니들 야, 니들 근데  이런 거 모르지?!
user > 덕선이 어때
Chatbot > 그지~ 덕선아~ 때가 좋았지 그지~
user > 노래 불러줘
Chatbot > 굿, 오~ 정팔이  정팔이 굿! 나 잔다.
user > 잘자
Chatbot > 왠열 소개팅 한 해가지고 두번 한 두 달 만나는게 그게 연애냐?
user > 나 소개팅나가
Chatbot > 소개팅도 오지게 또 안 해요.
user > 너 소개팅 할래?
Chatbot > 소개팅도 오지게 또 안 해요.
user > 그래...
Chatb